In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

打印出Dictionary数据类型

In [ ]:
!pip install xlrd
dict_all = pd.read_excel("/kaggle/input/GiveMeSomeCredit/Data Dictionary.xls")
dict_all

读入数据

In [ ]:
df_train = pd.read_csv("../input/GiveMeSomeCredit/cs-training.csv")
df_test = pd.read_csv("../input/GiveMeSomeCredit/cs-test.csv")

In [ ]:
df_train.info()

数据清洗

In [ ]:
df_train = df_train.rename({"Unnamed: 0":"ID"},axis = 1)
df_test = df_test.rename({"Unnamed: 0":"ID"},axis = 1)

In [ ]:
print(df_train.duplicated().value_counts())
print("------------------------")
print(df_test.duplicated().value_counts())

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

**解决空值问题**

In [ ]:
df_train.loc[df_train['age'] < 18]

In [ ]:
temp = df_train.copy()
temp.drop(65695,inplace = True)
temp.describe()

In [ ]:
df_train.loc[(df_train['age'] == 0),'age'] = df_train['age'].mean()
df_train.describe()

In [ ]:
df_train['NumberOfDependents'].fillna(int(df_train['NumberOfDependents'].mean()),inplace = True)

In [ ]:
df_test['NumberOfDependents'].fillna(int(df_test['NumberOfDependents'].mean()),inplace = True)

In [ ]:
df_train['MonthlyIncome'].fillna(int(df_train['MonthlyIncome'].mean()),inplace = True)

In [ ]:
df_test['MonthlyIncome'].fillna(int(df_test['MonthlyIncome'].mean()),inplace = True)

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

75%以上数据为不会遭受财务危机数据，分类并不均衡，画图展示

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure()
sns.countplot('SeriousDlqin2yrs',data = df_train)

**建立模型，使用梯度提升树**

划分训练集

In [ ]:
X = df_train.drop(['SeriousDlqin2yrs','ID'],axis = 1)
y = df_train['SeriousDlqin2yrs']
W = df_test.drop(['SeriousDlqin2yrs','ID'],axis = 1)
z = df_test['SeriousDlqin2yrs']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 111)

模型建立

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve,roc_auc_score
%matplotlib inline
def plot_roc_curve(fpr,tpr,label = None):
    plt.figure(figsize=(6,5))
    plt.plot(fpr,tpr,linewidth = 2,label = label)
    plt.plot([0,1],[0,1],"k--")
    plt.axis([0,1,0,1])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
gbc_clf = GradientBoostingClassifier(n_estimators = 300,learning_rate = 0.05,max_depth = 8,random_state = 112)
gbc_clf.fit(X_train, y_train)
gbc_clf_proba = gbc_clf.predict_proba(X_train)
gbc_clf_scores = gbc_clf_proba[:,1]
fpr_gbc,tpr_gbc,thresh_gbc = roc_curve(y_train,gbc_clf_scores)
plot_roc_curve(fpr_gbc,tpr_gbc)
print("AUC Score:",roc_auc_score(y_train,gbc_clf_scores))

交叉验证，并且调节参数

In [ ]:
gbc_clf_submission = GradientBoostingClassifier(n_estimators = 200,learning_rate = 0.05,max_depth = 4,random_state = 42)
gbc_clf_submission.fit(X_train,y_train)
gbc_clf_proba = gbc_clf_submission.predict_proba(X_train)
gbc_clf_scores = gbc_clf_proba[:,1]
gbc_val_proba = gbc_clf_submission.predict_proba(X_test)
gbc_val_scores = gbc_val_proba[:,1]
fpr_gbc,tpr_gbc,thresh_gbc = roc_curve(y_train,gbc_clf_scores)
plot_roc_curve(fpr_gbc,tpr_gbc)
print("AUC Score:",roc_auc_score(y_train,gbc_clf_scores))
print("AUC Score:",roc_auc_score(y_test,gbc_val_scores))

查看gbc对每个特征的重视程度

In [ ]:
def plot_feature_importances(model):
    plt.figure(figsize=(10,8))
    n_features = X.shape[1]
    plt.barh(range(n_features),model.feature_importances_,align = 'center')
    plt.yticks(np.arange(n_features),X.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1,n_features)
plot_feature_importances(gbc_clf)

In [ ]:
print(W.shape)
submission_proba = gbc_clf_submission.predict_proba(W)
submission_scores = submission_proba[:,1]
submission_scores.shape

In [ ]:
ids = np.arange(1,101504)
submission = pd.DataFrame({'Id':ids,'Probability':submission_scores})
submission.to_csv('submission.csv',index = False)